# 프로젝트 실습
> 진행 순서
1. 압축 파일 정리
2. 파일명 정리
3. 폴더 생성
4. 파일 분류 및 이동

## 1. 압축 파일 정리

In [4]:
# 정리 대상 폴더 경로 지정
target_path = './practice_folder'

In [1]:
import glob
import os

In [5]:
# 압축 파일 확인
zip_f_path = []

for filename in glob.glob(os.path.join(target_path, "**/*.zip"), recursive=True):
    zip_f_path.append(filename)
    print(zip_f_path)

['./practice_folder/센터가동현황_물류.zip']


In [2]:
import zipfile

* [📒 ZipFile.infolist()](https://docs.python.org/ko/3/library/zipfile.html#zipfile.ZipFile.infolist)

In [6]:
# 압축 파일 해제
for zip in zip_f_path:
    with zipfile.ZipFile(zip) as myzip:
        ## 아카이브의 각 멤버에 대한 ZipInfo 객체를 포함하는 리스트를 반환
        zip_info = myzip.infolist()
        for info in zip_info:
            ## 한글 깨짐 방지를 위한 decoding
            decoded = info.filename.encode('cp437').decode('euc-kr')
            info.filename = os.path.join(target_path, decoded)
            myzip.extract(info)

## 2. 파일명 정리

In [8]:
# # 파이썬에서 엑셀 파일을 다루는 라이브러리
# %pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 9.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


[📒 openpyxl](https://openpyxl.readthedocs.io/en/stable/)

In [9]:
import openpyxl as op

### 2-1. 파일명 수정 엑셀 파일을 생성하는 함수 작성
- 폴더별 파일명을 받아 엑셀 파일에 <i>[파일경로]</i>와 <i>[파일명(변경전)]</i>을 작성
- 바꾸고 싶은 파일명을 <i>[파일명(변경후)]</i> 컬럼에 작성
- 이후 [3단계](#3-파일명-변경하기)에서 해당 정보를 바탕으로 파일명을 변경

In [11]:
def getFileName(target_path):
    ## 여러 폴더가 있는 폴더 경로를 입력받아 list로 저장
    folders = os.listdir(target_path)

    # openpyxl Workbook 생성
    workbook = op.Workbook()

    # 생성한 Workbook의 활성화 시트를 정의
    worksheet = workbook.active

    ## 첫 row는 컬럼명을 기입해야 하므로 2행부터 시작
    i = 2
    ## 현재 폴더 위치
    current_path = target_path
    ## 현재 폴더의 파일들을 list로 저장
    files = os.listdir(current_path)

    ## 현재 폴더 위치의 파일 list를 for문으로 하나씩 접근
    for file in files:
        worksheet.cell(row=i, column=1).value = current_path + '/'
        worksheet.cell(row=i, column=2).value = file
        i += 1
    worksheet.cell(row=1, column=1).value = "파일경로"
    worksheet.cell(row=1, column=2).value = "파일명(변경전)"
    ## 이후 파일 이름을 변경한 후 저장할 컬럼명
    worksheet.cell(row=1, column=3).value = "파일명(변경후)"

    ## 위에서 작성한 파일을 저장
    workbook.save(os.path.join(target_path, "filelist.xlsx"))


In [13]:
getFileName(target_path)

## 3. 파일명 변경하기

### 3-1. 엑셀 파일을 읽어 수정할 파일명을 알아내는 함수 작성
- [2-1](#2-1-파일명-수정-엑셀-파일을-생성하는-함수-작성)의 결과로 작성된 엑셀 파일 읽기

#### [ ] list comprehension ?


In [22]:
# str형 filepath를 인수로 받아 list을 반환하는 함수
def readExcel(filepath : str) -> list:
    ## 파일 경로로부터 Workbook 객체 생성
    workbook = op.load_workbook(filepath)

    ## 활성화 worksheet 정의
    worksheet = workbook.active

    ## list comprehension 구문을 통해 각 컬럼의 데이터를 리스트화
    folderpath = [r[0].value for r in worksheet]
    file_before = [r[1].value for r in worksheet]
    file_after = [r[2].value for r in worksheet]
    
    ## 변경해야 할 파일 개수 확인
    len_num = len(folderpath)
    ## 결과값 저장할 리스트
    datalist = []

    ## for 문을 통해 튜플 작성
    for i in range(1, len_num):
        ## 각 행의 데이터를 저장하는 튜플
        temp = (folderpath[i], file_before[i], file_after[i])
        ## 리스트에 각 행의 값을 튜플 형태로 추가
        datalist.append(temp)

    return datalist


In [24]:
rename_list = readExcel(os.path.join(target_path, 'filelist.xlsx'))

### 3-2. 파일명을 수정하는 함수 작성
- [3-1](#3-1-엑셀-파일을-읽어-수정할-파일명을-알아내는-함수-작성)의 반환 결과를 인수로 파일명 수정하기

In [25]:
import shutil

In [26]:
def renameFiles(datalist : list):
    for data in datalist:
        print(f'{data[1]}의 파일명을 {data[2]}로 변경합니다.')
        shutil.move(data[0]+data[1], data[0]+data[2])

In [ ]:
renameFiles(rename_list)

## 4. 폴더 생성

### 4-1. 파일명의 분류 부분을 리스트화 하는 함수 작성
- `fnmatch`를 통해 정해진 패턴의 파일명에서 카테고리를 분류
- 분류된 카테고리를 기반으로 이후 폴더를 생성하여 파일을 카테고리별 폴더 하위에 저장

In [28]:
import fnmatch

In [29]:
def categorizeList(target_path : str) -> list :

    files = []

    for filename in os.listdir(target_path):
        ## 파일명의 끝이 _XXX(숫자 세 자리)로 끝나는 파일 탐색하여 리스트에 추가
        if fnmatch.fnmatch(filename, '*_[0-9][0-9][0-9].*'):
            files.append(filename)

    categories = []

    for file in files:
        ## 파일명을 '_'로 분리한 뒤
        temp = file.split('_')
        ## 그 중 -2 인덱싱 데이터를 리스트에 추가
        categories.append(temp[-2])

    ## 중복을 제거하기 위해 set 사용
    temp = set(categories)
    ## 다시 리스트로
    categories = list(temp)    

    return categories

In [32]:
# 위에서 포함되지 않는 것들을 저장하기 위한 '기타'도 리스트에 추가한다
categories = categorizeList(target_path) + ['기타']
categories

['생산', '인사', '물류', '재무', '마케팅', '기타']

### 4-2. 폴더 생성하는 함수 작성
- [4-1](#4-1-파일명의-분류-부분을-리스트화-하는-함수-작성)에서 생성한 카테고리 리스트를 통해 폴더를 생성

In [33]:
import pathlib

In [35]:
# 분류된 파일들을 저장할 최상위 폴더명
organized_path = './organized_folder'

In [36]:
def makeFolder(target_path : str, categories : list):
    for category in categories:
        ## 지정된 위치에 카테고리 이름대로 폴더 생성
        folder = pathlib.Path(os.path.join(target_path, category))
        folder.mkdir(parents=True, exist_ok=True)

In [37]:
makeFolder(organized_path, categories)

### 4-3. 파일 분류대로 이동시키는 함수 작성
- 기존 폴더의 파일들을 분류된 파일들을 카테고리별로 생성한 폴더로 이동

In [38]:
import shutil

In [41]:
def moveFiles(organized_path : str, target_path : str, categories : list):
    ## 이동시킬 파일명들을 리스트화
    files = os.listdir(target_path)
    
    category_dictionary = {}

    ## 파일명에 대한 폴더명을 딕셔너리에 저장
    for file in files:
        try:
            temps = file.split('_')
            ## 카테고리 리스트에 포함되지 않는다면, 에러 발생
            assert temps[-2] in categories
            ## 파일명 : 분류 형태로 딕셔너리에 추가
            category_dictionary[file] = temps[-2]
        except:
            ## 카테고리에 포함되지 않는다면 파일명 : '기타' 형태로 추가
            category_dictionary[file] = '기타'

    for key, value in category_dictionary.items():
        shutil.move(target_path+"/"+key, organized_path+"/"+value)

In [40]:
moveFiles(organized_path, target_path, categories)